## Table of Contents
### 1. Collaborative Filtering Algortihm
    0. Data Preprocessing
    1. Compute User Similairty Matrix
    2. Get Expected Rating with Similarity Matrix
    3. Recommend Top N Items
### 2. CF only with similar neighbors
    0. Data Preprocessing
    1. Compute User Similarity Matrix
    2. Find who the 'similar neighbors' are
    3. Get Expected Rating with Similarity Matrix
    4. Recommend Top N Items
    
### 3. CF with user bias
    1. 각 user의 평점 평균을 구함
    2. 각 item의 평점을 평점 편차로 수정(해당 평가를 내린 user의 평균과 얼마나 다른 지)
    3. 특정 user의 각 item에 대한 예상 평점 편차를 구함
    4. 해당 user의 예상 평점 편차에 그 user의 평균 평점을 더함.


### 4. IBCF

### + 평가지표
---

---

## Collaborative Filtering
: **취향**이 비슷한 사람들을 찾아, 그 사람들의 평가를 바탕으로 추천을 진행하자!

### Collaborative Filtering Algorithm
1. user간 평가의 **유사도** 계산
2. 유사도를 이용해 추천 대상이 되는 user가 평가하지 않은 모든 item에 대해 예상 평점을 구함(유사도의 가중 평균)
3. 예상 평가값이 높은 N개 추천

**User의 Similarity 계산** - ``sklearn.metrics.pairwise``
* 평점이 continuous인지(1~5점도 continuous로 생각), binary인지에 따라 평가 지표가 달라짐
* continuous: correlation, cosine similarity, euclidean 등
* binary: Jaccard

0. Data 준비

In [1]:
import pandas as pd
import numpy as np

# 본 문서에서 계속 사용할 데이터를 가져오는 method
def get_data():
    u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
    users = pd.read_csv('/Users/jisujung/Desktop/dev/RecSys/python을 이용한 개인화 추천시스템/data/u.user', sep='|', names=u_cols, encoding='latin-1')
    i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
              'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
              'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
              'Thriller', 'War', 'Western']
    movies = pd.read_csv('/Users/jisujung/Desktop/dev/RecSys/python을 이용한 개인화 추천시스템/data/u.item', sep='|', names=i_cols, encoding='latin-1')
    r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_csv('/Users/jisujung/Desktop/dev/RecSys/python을 이용한 개인화 추천시스템/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
    
    return users, movies, ratings

In [10]:
users, movies, ratings= get_data()

In [4]:
users.head(2)

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043


In [5]:
movies.head(2)

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
ratings.head(2)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [11]:
# 전처리
ratings= ratings.drop('timestamp', axis=1)
movies= movies[['movie_id', 'title']]

In [12]:
# train_test_split
# user_id를 train/test set에 고르게 담기 위해 stratify
from sklearn.model_selection import train_test_split
x= ratings.copy()
y= ratings['user_id']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.25, stratify= y)

In [13]:
# 평가 지표 및 평가 method 정의
def RMSE(y_pred, y_true):
    return np.sqrt(np.mean((np.array(y_pred) - np.array(y_true))**2))

def score(model):
    id_pairs= zip(x_test['user_id'], x_test['movie_id'])
    y_pred= np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true= np.array(x_test['rating'])
    return RMSE(y_pred, y_true)

1. **similarity 계산**

similarity 계산을 위해서는 Nan값을 fillna(0)로 **채워야 한다**.
    -> 이렇게 되면 같은 것을 안 본 사람들에 대한 유사도가 약간 높아지기는 하겠다..?

In [15]:
rating_matrix= x_train.pivot(index= 'user_id', columns= 'movie_id', values= 'rating')

In [17]:
rating_matrix.head(2)

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy= rating_matrix.copy().fillna(0)
user_similarity= cosine_similarity(matrix_dummy)
user_similarity= pd.DataFrame(user_similarity, index= rating_matrix.index, columns= rating_matrix.index)

2. **similairity를 이용하여 가중평균 rating 계산**


- 모든 추천시스템에서 그렇듯이, 평가하고자 하는 item이 기존의 item matrix에 있는 지 확인해보아야 함(아무도 구매/평가하지 않았던 item은 추천이 힘듬). 없다면 1)아에 추천 목록에서 제외하거나, 2)중간 점수 등으로 대체하거나. 여기서는 2)의 방법을 택함
- 가중평균을 계산하기 위해, 어떤 user가 해당 item을 평가하고 어떤 user는 평가하지 않았는 지를 구분할 필요가 있음. 평가하지 않은 user의 경우 가중평균 계산에서 제외 해야함.
- '평가하지 않음' 자체를 어떤 aciton으로 추정 할 수도 있을듯. 예를 들어, 나와 유사도가 높은 사람들이 특정 영화를 보지 않았다면... 별로 땡기지 않는 것 아닐까? - similarity가 k 이상인 user들 중 unrated의 비중이 n% 이상인 item은 추천에서 제외한다. 등의 rule을 만들 수도 있을 듯

In [18]:
def movie_cf(user_id, movie_id):
    if movie_id in rating_matrix:
        similarity= user_similarity[user_id].copy()
        movie_ratings= rating_matrix[movie_id].copy()
        
        none_rated_user= movie_ratings[movie_ratings.isnull()].index
        
        movie_ratings= movie_ratings.dropna()
        similarity= similarity.drop(none_rated_user)
        
        mean_rating= np.dot(movie_ratings, similarity) / np.sum(similarity)
        
    else:
        mean_rating= 3.0
        
    return mean_rating

In [19]:
score(movie_cf)

1.014572385439244

---
## CF with similar neighbors
추천 대상인 user와 성향이 비슷한 user만 고려하면 어떨까?

**비슷한 user의 정의**
1. KNN
2. Thresholding: 일정 값 이상의 similarity를 가진 user들만 neighbor로 인정

보통은 thresholding이 KNN보다 정확하지만, 특정 similarity를 넘는 user가 없어 추천을 할 수 없는 경우가 잦으므로 KNN도 많이 사용된다.

### KNN을 이용한 CF Algorithm
1. 특정 item의 평가 데이터를 가져온다.
2. neighbor의 size에 따라,
    * neighbor_size == 0
        * similarity를 기준으로 User 평가 데이터의 가중 평균을 구한다(basic CF)
    * neighbor_size > 0
        * 해당 item을 평가한 user가 2명 이상인 경우
            1. neighbor size를 확인한다.
            2. 유사도를 순서대로 정렬하여,neighbor size만큼 similar한 user의 평점을 가져온다.
            3. neighbor의 평가 데이터의 가중 평균을 구한다
        * 해당 item을 평가한 user가 1명 이하인 경우
            1. 평점 - 중간 점수

전처리 과정과 유사도 matrix를 구하는 과정은 기본 알고리즘과 같음

In [21]:
users, movies, ratings= get_data()
ratings= ratings.drop('timestamp', axis=1)
movies= movies[['movie_id', 'title']]

from sklearn.model_selection import train_test_split
x= ratings.copy()
y= ratings['user_id']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.25, stratify= y)

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

rating_matrix= x_train.pivot(columns= 'movie_id', index= 'user_id', values= 'rating')

# neigbor_size=0 denotes normal CF(using every users data)
def score(model, neighbor_size=0):
    id_pairs= zip(x_test['user_id'], x_test['movie_id'])
    y_pred= np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true= np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

**Main Algorithm**

In [23]:
def cf_knn(user_id, movie_id, neighbor_size= 0):
    if movie_id in rating_matrix:
        sim_scores= user_similarity[user_id].copy()
        movie_ratings= rating_matrix[movie_id].copy()
        
        none_rating_idx= movie_ratings[movie_ratings.isnull()].index
        
        sim_scores= sim_scores.drop(none_rating_idx)
        movie_ratings= movie_ratings.drop(none_rating_idx)
        
        if neighbor_size== 0:
            mean_rating= np.dot(sim_scores, movie_ratings) / np.sum(sim_scores)
            
        else:
            if len(sim_scores) > 1:
                neighbor_size= min(neighbor_size, len(sim_scores))
                
                sim_scores= np.array(sim_scores)
                movie_ratings= np.array(movie_ratings)
                
                user_idx= np.argsort(sim_scores)
                movie_ratings= movie_ratings[user_idx][-neighbor_size:]
                sim_scores= sim_scores[user_idx][-neighbor_size:]
                
                mean_rating= np.dot(sim_scores, movie_ratings) / np.sum(sim_scores)
            
            else:
                mean_rating= 3.0
    else:
        mean_rating= 3.0
        
    return mean_rating

In [24]:
score(cf_knn, neighbor_size= 30)

1.0117610371448242

### Actual Recommending
* Input: 추천을 받을 user ID, 추천 받을 item 수(n), 이웃 크기
* Output: n개의 추천 item list

In [27]:
def recom_movie(user_id, n_items= 5, neighbor_size= 30):
    user_movie= rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie]= 0
        else:
            user_movie.loc[movie]= cf_knn(user_id, movie, neighbor_size)
    
    movie_sort= user_movie.sort_values(ascending=False)[:n_items]
    recom_movies= movies.loc[movie_sort.index]
    
    recommendations= recom_movies['title']
    
    return recommendations

In [28]:
recom_movie(2)

movie_id
1189                              That Old Feeling (1997)
1467                                     Cure, The (1995)
119                                     Striptease (1996)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1293                     Ayn Rand: A Sense of Life (1997)
Name: title, dtype: object

---
## CF with user bias
특정 user는 점수를 잘 주는 경향이 있고, 다른 user는 그렇지 않다.

즉, 같은 3점이 누군가에게는 칭찬이고, 누군가에게는 비난이다.

따라서 절대적인 점수보다 **평점 편차**(각 user의 평점 평균과의 차이)를 이용해야 한다.

전처리 과정과 유사도 matrix를 구하는 과정은 기본 알고리즘과 같음

In [32]:
users, movies, ratings= get_data()

ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

from sklearn.model_selection import train_test_split
x= ratings.copy()
y= ratings['user_id']
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.25, stratify= y)

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_test)) ** 2))

rating_matrix= x_train.pivot(columns= 'movie_id', index= 'user_id', values= 'rating')

def score(model, neighbor_size= 0):
    id_pairs= zip(x_test['user_id'], x_test['movie_id'])
    y_pred= np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true= np.array(x_test['rating'])    
    return RMSE(y_true, y_pred)

from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy= rating_matrix.copy().fillna(0)
user_similarity= cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity= pd.DataFrame(user_similarity, index= rating_matrix.index, columns= rating_matrix.index)

### CF wih user bias Algorithm
1. 각 user의 평점 평균을 구함
2. 각 item의 평점을 평점 편차로 수정(해당 평가를 내린 user의 평균과 얼마나 다른 지)
3. 특정 user의 각 item에 대한 예상 평점 편차를 구함
4. 해당 user의 예상 평점 편차에 그 user의 평균 평점을 더함.

In [33]:
rating_mean= rating_matrix.mean(axis=1)
rating_bias= (rating_matrix.T - rating_mean).T

In [42]:
def cf_knn_bias(user_id, movie_id, neighbor_size= 0):
    if movie_id in rating_bias:
        sim_scores= user_similarity[user_id].copy()
        movie_ratings= rating_bias[movie_id].copy()
        
        none_rating_idx= movie_ratings[movie_ratings.isnull()].index
        movie_ratings= movie_ratings.drop(none_rating_idx)
        sim_scores= sim_scores.drop(none_rating_idx)
        
        if neighbor_size == 0:
            prediction= np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            prediction+= rating_mean[user_id]
        
        else:
            if len(sim_scores) > 1:
                neighbor_size= min(neighbor_size, len(sim_scores))
                sim_scores= np.array(sim_scores)
                movie_ratings= np.array(movie_ratings)
                
                user_idx= np.argsort(sim_scores)
                
                sim_scores= sim_scores[user_idx][-neighbor_size:]
                movie_ratings= movie_ratings[user_idx][-neighbor_size:]
                
                prediction=  np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction+= rating_mean[user_id]
                
            else:
                prediction= rating_mean[user_id]
                
    else:
        prediction= rating_mean[user_id]
        
    return prediction

In [43]:
score(cf_knn_bias, 20)

530.7466511999863

## IBCF: Item-Based Collaborative Filtering
user들의 평가 패턴을 바탕으로 **item간의 유사도**를 계산

구체적으로
* 예측 대상 user가 평가한 item의 평점과 다른 item과의 유사도를 가중하여 계산

In [44]:
users, movies, ratings= get_data()

ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

In [45]:
def IBCF(user_id, movie_id):
    if movie_id in item_similarity:
        sim_scores= item_similarity[movie_id]
        user_rating= rating_matrix_t[user_id]
        
        none_rating_idx= user_rating[user_rating.isnull()].index
        
        user_rating= user_rating.dropna()
        sim_scores= sim_scores.drop(none_rating_idx)
        
        mean_rating= np.dot(sim_scores, user_rating) / sim_scores.sum()
        
    else:
        mean_rating= 3.0
    
    return mean_rating

In [46]:
score(IBCF)

1.00705243442935

## 각종 평가지표들
주로 Accuracy를 사용하지만, 마치 머신러닝처럼 다양한 평가 metric이 사용된다.

* precision
* recall
* coverage
* F1 score 등등!